In [1]:
import numpy as np

La fonction gibbs_step_param exécute une seule itération de l'algorithme d'échantillonnage de Gibbs, qui consiste à générer des échantillons à partir de la loi a posteriori des paramètres et à les mettre à jour sur la base de la distance minimale entre les échantillons et les données observées.

In [2]:
    """
    Paramètres :
    - data (numpy.ndarray) : Données observées.
    - hyper (float) : Valeur initiale de l'hyperparamètre.
    - var (float) : Variance des paramètres.
    - sigm (float) : Écart-type des observations.
    - nbeps (int) : Nombre d'échantillons pour les paramètres.
    - qq (float) : quantile pour la tolérance de l'algorithme.

    Return :
    - thetc (numpy.ndarray) : Paramètres mis à jour après l'étape de Gibbs.
    - dists (float) : Somme des distances minimales entre les observations et les simulations.
    """

def gibbs_step_param(data, hyper, var, sigm, nbeps, qq):
    # Étape de Gibbs pour la mise à jour des paramètres
    p = len(data)
    thetc = np.zeros(p)
    dists = 0
    
    for i in range(p):
        # Générer des échantillons pour les paramètres
        thettest = np.random.normal(hyper, var, nbeps)
        # Calculer la moyenne des échantillons pour chaque observation
        test = np.mean(np.random.normal(thettest[:, None], sigm, (nbeps, len(data))), axis=0)
        # Calculer la distance entre les observations et les simulations
        dist = np.abs(test - data[i])
        # Mettre à jour les paramètres en choisissant la simulation avec la plus petite distance
        thetc[i] = thettest[np.argmin(dist)]
        dists += np.min(dist)
    
    return thetc, dists

La fonction gibbhyper met à jour l'hyperparamètre en utilisant l'algorithme d'échantillonnage de Gibbs. Elle suppose une loi a priori uniforme pour l'hyperparamètre.

In [3]:
"""
    Paramètres :
    - thet (numpy.ndarray) : Un tableau contenant les valeurs actuelles des paramètres.
    - nbeps2 (int) : Le nombre d'échantillons à générer pour l'hyperparamètre.
    - var (float) : La variance des valeurs des paramètres.

    Return : 
    - res (numpy.ndarray) : Un tableau contenant les échantillons générés pour l'hyperparamètre.
    - dist (float) : La distance minimale entre les échantillons générés et la moyenne des paramètres.
"""

def gibbshyper(thet, nbeps2, var):
    # Étape de Gibbs pour la mise à jour de l'hyperparamètre, avec loi a priori uniforme
    res = np.random.uniform(-4, 4, nbeps2)
    # Calculer la moyenne des échantillons pour chaque paramètre
    test = np.mean(np.random.normal(res[:, None], var, (nbeps2, len(thet))), axis=0)
    # Calculer la distance entre la moyenne des échantillons et la moyenne des paramètres
    dist = np.abs(test - np.mean(thet))
    # Mettre à jour l'hyperparamètre en choisissant la simulation avec la plus petite distance
    return res[np.argmin(dist)], np.min(dist)


La fonction gibbs_abc met en œuvre l'algorithme ABC de Gibbs, qui met à jour les paramètres et l'hyperparamètre de manière itérative. Elle suppose une distribution a priori uniforme pour l'hyperparamètre.

In [9]:

def gibbs_abc(data, thetini, hyperini, sigm, var, nbeps1, nbeps2, nbpts, qq):
    # Algorithme complet de Gibbs ABC
    reshyper = np.zeros(nbpts + 1)
    resparam = np.zeros((len(thetini), nbpts + 1))
    reshyper[0] = hyperini
    resparam[:, 0] = thetini
    resdist = np.zeros(nbpts)
    
    for i in range(1, nbpts + 1):
        resdist[i - 1] = 0
        # Mettre à jour les paramètres
        VV, dist_param = gibbs_step_param(data, reshyper[i - 1], var, sigm, nbeps1, qq)
        resparam[:, i] = VV
        resdist[i - 1] += dist_param
        # Mettre à jour l'hyperparamètre
        WW, dist_hyper = gibbshyper(resparam[:, i], nbeps2, var)
        reshyper[i] = WW
    
    return reshyper, resparam, resdist


In [10]:

# Exemple d'utilisation :
data = np.array([3.5, 4.2, 5.1, 6.3])  # données observées
thetini = np.array([1.0, 2.0, 3.0, 4.0])  # valeurs initiales des paramètres
hyperini = 0.0  # valeur initiale de l'hyperparamètre
sigm = 0.1  # écart type des observations
var = 0.5  # variance des paramètres
nbeps1 = 1000  # nombre d'échantillons pour les paramètres
nbeps2 = 1000  # nombre d'échantillons pour l'hyperparamètre
nbpts = 10  # nombre d'itérations de l'algorithme
qq = 0.9  # quantile pour la tolérance

# Appel de l'algorithme Gibbs ABC
reshyper, resparam, resdist = gibbs_abc(data, thetini, hyperini, sigm, var, nbeps1, nbeps2, nbpts, qq)

# Affichage des résultats
print("Estimations des paramètres :", resparam[:, -1])
print("Estimation de l'hyperparamètre :", reshyper[-1])
print("Distances calculées à chaque itération :", resdist)


Estimations des paramètres : [-3.7279946  -3.96611515 -3.52297185 -4.44193806]
Estimation de l'hyperparamètre : -1.8925754426362902
Distances calculées à chaque itération : [19.10195956 27.24648106 34.15857368 34.16378373 22.26085035 25.89245247
  8.48020515 12.23388897 16.36415865 34.09458773]


In [15]:
import numpy as np

# Chargement des données
toydata = np.genfromtxt("toydata.csv")
toyparam = np.genfromtxt("toyparam.csv", skip_header=1)
toyparamsig = np.genfromtxt("toyparamsig.csv", skip_header=1)


ValueError: Some errors were detected !
    Line #2 (got 6 columns instead of 5)
    Line #3 (got 6 columns instead of 5)
    Line #4 (got 6 columns instead of 5)
    Line #5 (got 6 columns instead of 5)
    Line #6 (got 6 columns instead of 5)
    Line #7 (got 6 columns instead of 5)
    Line #8 (got 6 columns instead of 5)
    Line #9 (got 6 columns instead of 5)
    Line #10 (got 6 columns instead of 5)
    Line #11 (got 6 columns instead of 5)
    Line #12 (got 6 columns instead of 5)
    Line #13 (got 6 columns instead of 5)
    Line #14 (got 6 columns instead of 5)
    Line #15 (got 6 columns instead of 5)
    Line #16 (got 6 columns instead of 5)
    Line #17 (got 6 columns instead of 5)
    Line #18 (got 6 columns instead of 5)
    Line #19 (got 6 columns instead of 5)
    Line #20 (got 6 columns instead of 5)
    Line #21 (got 6 columns instead of 5)
    Line #22 (got 6 columns instead of 5)
    Line #23 (got 6 columns instead of 5)
    Line #24 (got 6 columns instead of 5)
  

In [14]:
toydata

array([[            nan,  1.51976308e+00, -1.51113973e-01,
        -5.00753803e+00, -2.70502410e+00, -1.31271577e+00],
       [            nan, -1.53073437e+00, -5.43992192e-01,
         1.71903438e+00, -2.11156849e+00,  3.21823438e+00],
       [            nan,  1.26555815e+00, -8.06111087e-01,
        -4.94357941e+00, -2.33946856e+00, -6.04782556e-01],
       [            nan, -1.95505714e+00, -3.08354851e-01,
        -6.63539534e+00,  7.12320670e-01, -2.16231092e+00],
       [            nan, -4.68701665e-01,  2.94669471e-01,
        -1.49202579e+00, -2.72035179e-01, -3.57305493e-01],
       [            nan,  2.43061235e-01,  5.99281058e-01,
         4.06222732e-01, -3.85241268e+00,  5.31085588e-01],
       [            nan, -3.52354675e+00, -7.63037434e-02,
         6.67499854e+00, -1.91269794e+00,  2.31950696e+00],
       [            nan, -4.01015802e+00, -1.72349331e-01,
         1.92535640e+00, -1.93799246e+00,  3.64663683e-01],
       [            nan,  4.08310400e+00, -3.931

In [ ]:

# Sélection des colonnes appropriées
data = toydata[:, 1]  # Sélection de la deuxième colonne des données
thetini = toyparam[:, 0]  # Première colonne des paramètres initiaux des moyennes µ
hyperini = toyparam[0, 1]  # Deuxième colonne des paramètres initiaux α
sigm = toyparamsig[0]  # Écart-type des observations (σ)
var = toyparamsig[1]**2  # Variance des paramètres (σ^2)

# Autres paramètres
nbeps1 = 1000  # Nombre d'échantillons pour les paramètres (Nα)
nbeps2 = 1000  # Nombre d'échantillons pour l'hyperparamètre (Nµ)
nbpts = 10  # Nombre d'itérations de l'algorithme Gibbs
qq = 0.9  # Quantile pour la tolérance de l'algorithme

# Définition des fonctions (les mêmes que précédemment)

# Application de l'algorithme Gibbs ABC
reshyper, resparam, resdist = gibbs_abc(data, thetini, hyperini, sigm, var, nbeps1, nbeps2, nbpts, qq)

# Affichage des résultats
print("Estimations des paramètres à la dernière itération :", resparam[:, -1])
print("Estimation de l'hyperparamètre à la dernière itération :", reshyper[-1])
print("Distances calculées à chaque itération :", resdist)
